```
print("Size of adjacency matrix = " + str(adj.shape))
print("Size of feature matrix = " + str(features.shape))     # expected to be sparse
print("Size of label matrix = " + str(train_mask.shape))
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(train_mask.shape)
print(val_mask.shape)
```
produces these results:
```
Size of adjacency matrix = (2708, 2708)
Size of feature matrix = (2708, 1433)
Size of label matrix = (2708,)
(2708, 7)
(2708, 7)
(2708, 7)
(2708,)
(2708,)
```

A mask is used which is why they are all the same size

## Testing Hamming loss function

In [75]:
# Short demonstation of how Hamming loss will be calculated for multiple vectors
import tensorflow as tf

# define a threshold
thresh = tf.constant(0.5)
sample_output = tf.constant([[0.23, 0.98, 0.78, 0.52, 0.48], [0.1, 0.2, 0.3, 0.6, 0.9], [
                     0.9, 0.1, 0.99, 0.0, 0.03]])      # sample logit output
# prediciton based on output and threshold
preds = tf.math.greater(sample_output, thresh, name=None)
print(preds)

# define samplel labels
labels = tf.constant([[0, 1, 1, 0, 1], [0, 0, 0, 0, 1], [1, 0, 1, 0, 0]])
bool_labels = tf.equal(labels, tf.constant(
    1))               # transform to bool
print(bool_labels)

tf.Tensor(
[[False  True  True  True False]
 [False False False  True  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
tf.Tensor(
[[False  True  True False  True]
 [False False False False  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)


In [77]:
# trying to fix pre masking hamming loss shape
def masked_hamming_loss(outputs, labels, mask):
    """hamming loss with masking."""
    # make prediction
    # define a threshold
    thresh = tf.constant(0.5)
    # prediciton based on output and threshold
    preds = tf.math.greater(outputs, thresh, name=None)

    print("Predictions")
    print(preds)
   
    bool_labels = tf.equal(tf.cast(labels, tf.int32),
                           tf.constant(1))  # transform to bool

    print("Labels")
    print(bool_labels)

    # find where there are mistakes
    exclusive_or = tf.math.not_equal(bool_labels, preds)
    print(exclusive_or)

    # count how many true in exclusive or of label (how many are wrong)
    ham_loss_all = tf.math.reduce_mean(
        tf.cast(exclusive_or, tf.float32), axis=1)
        
    print("Hamming Loss before mask")
    print(ham_loss_all)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(input_tensor=mask)
    ham_loss_all *= mask
    return tf.reduce_mean(input_tensor=ham_loss_all)


print(masked_hamming_loss(sample_output, labels, mask))

Predictions
tf.Tensor(
[[False  True  True  True False]
 [False False False  True  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
Labels
tf.Tensor(
[[False  True  True False  True]
 [False False False False  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
tf.Tensor(
[[False False False  True  True]
 [False False False  True False]
 [False False False False False]], shape=(3, 5), dtype=bool)
Hamming Loss before mask
tf.Tensor([0.4 0.2 0. ], shape=(3,), dtype=float32)
tf.Tensor(0.3, shape=(), dtype=float32)


## Testing Alpha Eval

In [86]:
# Short demonstation of how alpha eval loss will be calculated for multiple vectors
import tensorflow as tf

# define a threshold
thresh = tf.constant(0.5)
sample_output = tf.constant([[0.23, 0.98, 0.78, 0.52, 0.48], [0.1, 0.2, 0.3, 0.6, 0.9], [
    0.9, 0.1, 0.99, 0.0, 0.03]])      # sample logit output
# prediciton based on output and threshold
preds = tf.math.greater(sample_output, thresh, name=None)
print("Predited labels")
print(preds)

# define samplel labels
labels = tf.constant([[0, 1, 1, 0, 1], [0, 0, 0, 0, 1], [1, 0, 1, 0, 0]])
bool_labels = tf.equal(labels, tf.constant(
    1))               # transform to bool
print("True labels")
print(bool_labels)

# find where there are mistakes
exclusive_or = tf.math.not_equal(bool_labels, preds)
print("True if mistake")
print(exclusive_or)

# find all labels
print("True if there is a label in predicted or true set")
union = tf.math.logical_or(bool_labels, preds)
print(union)

# divide
print(tf.reduce_sum(tf.cast(exclusive_or, dtype=tf.int32), axis=1))
print(tf.reduce_sum(tf.cast(union, dtype=tf.int32), axis=1))
alpha_eval = tf.divide_no_nan(tf.reduce_sum(tf.cast(exclusive_or, dtype=tf.int32), axis=1),
          tf.reduce_sum(tf.cast(union, dtype=tf.int32), axis=1))

print(alpha_eval)


Predited labels
tf.Tensor(
[[False  True  True  True False]
 [False False False  True  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
True labels
tf.Tensor(
[[False  True  True False  True]
 [False False False False  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
True if mistake
tf.Tensor(
[[False False False  True  True]
 [False False False  True False]
 [False False False False False]], shape=(3, 5), dtype=bool)
True if there is a label in predicted or true set
tf.Tensor(
[[False  True  True  True  True]
 [False False False  True  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
tf.Tensor([2 1 0], shape=(3,), dtype=int32)
tf.Tensor([4 2 2], shape=(3,), dtype=int32)
tf.Tensor([0.5 0.5 0. ], shape=(3,), dtype=float64)


## Testing Loss

In [32]:
def masked_softmax_cross_entropy(preds, labels):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.sigmoid_cross_entropy_with_logits(
        logits=preds, labels=tf.cast(labels,dtype=tf.float32))
    print(loss)
    loss = tf.nn.softmax_cross_entropy_with_logits(
        logits=preds, labels=tf.cast(labels, dtype=tf.float32))
    print(loss)
    #mask = tf.cast(mask, dtype=tf.float32)
    #mask /= tf.reduce_mean(mask)
    #loss *= mask
    return tf.reduce_mean(loss)


masked_softmax_cross_entropy(sample_output, labels) # sigmoid loss is returning a matrix but softmax loss returns a vector


tf.Tensor(
[[0.8147452  0.31867993 0.3773441  0.9865731  0.48167482]
 [0.7443967  0.79813886 0.8543552  1.037488   0.34115386]
 [0.34115386 0.7443967  0.31596094 0.6931472  0.70825964]], shape=(3, 5), dtype=float32)
tf.Tensor([4.482969  1.1741972 2.3392463], shape=(3,), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=2.6654708>

In [26]:
mask = tf.constant([1, 1, 0])

# with predictions
def masked_sigmoid_cross_entropy(outputs, labels, mask):
    """Softmax cross-entropy loss with masking."""
    # define a threshold
    thresh = tf.constant(0.5)
    # prediciton based on output and threshold
    preds = tf.math.greater(outputs, thresh, name=None)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(
        logits=tf.cast(preds, dtype=tf.float32), labels=tf.cast(labels, dtype=tf.float32))

    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)

    loss = tf.multiply(loss, tf.reshape(mask, (-1, 1)))
    return tf.reduce_mean(loss)


# sigmoid loss is returning a matrix but softmax loss returns a vector
print(masked_sigmoid_cross_entropy(sample_output, labels, mask))

tf.Tensor(0.70320445, shape=(), dtype=float32)


In [25]:
mask = tf.constant([1, 1, 0])

# with OUT predictions
def masked_sigmoid_cross_entropy(outputs, labels, mask):
    """Softmax cross-entropy loss with masking."""

    loss = tf.nn.sigmoid_cross_entropy_with_logits(
        logits=tf.cast(outputs, dtype=tf.float32), labels=tf.cast(labels, dtype=tf.float32))

    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)

    loss = tf.multiply(loss, tf.reshape(mask, (-1, 1)))
    return tf.reduce_mean(loss)


# sigmoid loss is returning a matrix but softmax loss returns a vector
print(masked_sigmoid_cross_entropy(sample_output, labels, mask))


tf.Tensor(0.675455, shape=(), dtype=float32)
